In [19]:
import psycopg2
import pandas as pd
# Connection parameters, yours will be different
param_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "postgres"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [20]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df1 = pd.DataFrame(tupples, columns=column_names)
    return df1

In [21]:
# Connect to the database
conn = connect(param_dic)
column_names = ['date','price','open','high','low','volume','change','value']
# Execute the "SELECT *" query
df1 = postgresql_to_dataframe(conn, "select * from btcjoin", column_names)
df1.head()
df1.to_csv('joined.csv')

Connecting to the PostgreSQL database...
Connection successful


In [22]:
import pandas as pd 
df = pd.read_csv("joined.csv",index_col='date', parse_dates=['date'])
df = df.drop(columns=['Unnamed: 0'])
df = df.sort_index()
df

,price,open,high,low,volume,change,value
date,,,,,,,
2010-09-25,0.1,0.1,0.1,0.1,2.15K,0.00%,2
2010-09-26,0.1,0.1,0.1,0.1,12.06K,0.00%,2
2010-09-27,0.1,0.1,0.1,0.1,10.75K,0.00%,2
2010-09-28,0.1,0.1,0.1,0.1,7.09K,0.00%,2
2010-09-29,0.1,0.1,0.1,0.1,23.48K,0.00%,2
...,...,...,...,...,...,...,...
2022-10-21,"19,162.60","19,042.90","19,245.50","18,703.30",294.66K,0.63%,84879615
2022-10-22,"19,204.80","19,162.60","19,249.90","19,116.10",118.24K,0.22%,84888147
2022-10-23,"19,571.20","19,204.80","19,680.90","19,092.50",180.63K,1.91%,84893809


In [23]:
df2 = pd.read_csv("activeadd.csv", parse_dates=['date'])
df2['date'] = df2['date'].dt.strftime('%Y-%m-%d')
df2["date"] = df2["date"].astype("datetime64[ns]")
df2["address"] = df2["address"].astype("float")
df2 = df2.set_index('date')
df2

,address
date,
2009-01-03,0.0
2009-01-04,0.0
2009-01-05,0.0
2009-01-06,0.0
2009-01-07,0.0
...,...
2022-10-27,897118.0
2022-10-28,938666.0
2022-10-29,879976.0


In [24]:
df3 = pd.read_csv("mined.csv", parse_dates=['date'])
df3['date'] = df3['date'].dt.strftime('%Y-%m-%d')
df3["date"] = df3["date"].astype("datetime64[ns]")
df3["mined"] = df3["mined"].astype("float")
df3 = df3.set_index('date')
df3 = df3.sort_index()
df3

,mined
date,
2009-01-02,50.0
2009-01-03,50.0
2009-01-04,50.0
2009-01-05,50.0
2009-01-06,50.0
...,...
2022-10-27,19191000.0
2022-10-28,19192125.0
2022-10-29,19193025.0


In [25]:
import yfinance as yf
result = pd.merge(df, df2, left_index=True, right_index=True)
result = pd.merge(result, df3, left_index=True, right_index=True)
result = result.reset_index()
result.to_csv('btcjoin.csv')

